In [3]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

food_url = 'https://tarkov-market.com/tag/food'
drink_url = 'https://tarkov-market.com/tag/drinks'
# URL to be scraped
food_page = requests.get(food_url)
drink_page = requests.get(drink_url)


food_soup = BeautifulSoup(food_page.content, 'html.parser')
drink_soup = BeautifulSoup(drink_page.content, 'html.parser')


In [4]:
food_table_list_div = food_soup.find('div', class_='table-list')
drink_table_list_div = drink_soup.find('div', class_='table-list')

In [5]:
provisions_price_dict = {}

In [7]:
all_food_rows = food_table_list_div.find_all('div', class_='row')
all_drink_rows = drink_table_list_div.find_all('div', class_='row')

    # Iterate over each child div
for child_div in all_food_rows[1:]:
    # Do something with each child div
    #print(child_div.prettify())
    price = child_div.find('span', class_='price-main')
    price_num = re.sub(r'\D', '', price.text)
    print(price_num)
    print(price.text)
    name = child_div.find('span', class_='name').text.replace('"', '')
    print(name)
    print('-------------------')
    if name == 'Bottle of Tarkovskaya vodka (bad)':
        continue
    provisions_price_dict[name] = int(price_num)

for child_div in all_drink_rows[1:]:
    # Do something with each child div
    #print(child_div.prettify())
    price = child_div.find('span', class_='price-main')
    price_num = re.sub(r'\D', '', price.text)
    print(price_num)
    print(price.text)
    name = child_div.find('span', class_='name').text.replace('"', '')
    print(name)
    print('-------------------')
    if name == 'Bottle of Tarkovskaya vodka (bad)':
        continue
    provisions_price_dict[name] = int(price_num)
    
    

25246
25,246₽
Can of squash spread
-------------------
15435
15,435₽
Can of green peas
-------------------
17452
17,452₽
Rye croutons
-------------------
23179
23,179₽
Pack of oat flakes
-------------------
9030
9,030₽
Emelya rye croutons
-------------------
12203
12,203₽
Can of pacific saury
-------------------
22694
22,694₽
Can of beef stew (Large)
-------------------
11500
11,500₽
Jar of DevilDog mayo
-------------------
11515
11,515₽
Can of beef stew (Small)
-------------------
23209
23,209₽
Iskra ration pack
-------------------
14529
14,529₽
Can of humpback salmon
-------------------
11958
11,958₽
Pack of Tarker dried meat
-------------------
15905
15,905₽
Can of condensed milk
-------------------
28797
28,797₽
Pack of sugar
-------------------
14073
14,073₽
Army crackers
-------------------
18079
18,079₽
Salty Dog beef sausage
-------------------
13864
13,864₽
Can of sprats
-------------------
9955
9,955₽
Slickers chocolate bar
-------------------
7641
7,641₽
Pack of instant nood

In [8]:
from items_dict import items_dict

for item in items_dict.items():
    print(item)

('Emelya rye croutons', (10, -10))
('Army crackers', (10, -5))
('Rye croutons', (20, -10))
('Slickers chocolate bar', (30, -15))
('Alyonka chocolate bar', (35, -15))
('Pack of oat flakes', (40, -13))
('Pack of sugar', (70, -45))
('Can of condensed milk', (75, -65))
('Jar of DevilDog mayo', (100, -99))
('Can of beef stew (Small)', (45, -5))
('Can of herring', (47, -3))
('Can of pacific saury', (48, -2))
('Can of beef stew (Large)', (50, -5))
('Can of humpback salmon', (50, -5))
('Salty Dog beef sausage', (50, -5))
('Can of sprats', (55, -5))
('Can of green peas', (35, 5))
('Pack of Tarker dried meat', (55, -5))
('Pack of instant noodles', (25, -10))
('Can of squash spread', (40, 0))
('MRE ration pack', (70, 0))
('Iskra ration pack', (80, 0))
('Can of Ice Green tea', (0, 15))
('Can of TarCola soda', (5, 15))
('Pack of apple juice', (5, 25))
('Can of Hot Rod energy drink', (10, 20))
('Can of Max Energy energy drink', (12, 20))
('Pack of Vita juice', (8, 28))
('Pack of Grand juice', (6, 30

In [13]:
for key in items_dict:
    if key not in provisions_price_dict:
        print(key)

Can of herring
MRE ration pack


In [9]:
print (provisions_price_dict)

{'Can of squash spread': 25246, 'Can of green peas': 15435, 'Rye croutons': 17452, 'Pack of oat flakes': 23179, 'Emelya rye croutons': 9030, 'Can of pacific saury': 12203, 'Can of beef stew (Large)': 22694, 'Jar of DevilDog mayo': 11500, 'Can of beef stew (Small)': 11515, 'Iskra ration pack': 23209, 'Can of humpback salmon': 14529, 'Pack of Tarker dried meat': 11958, 'Can of condensed milk': 15905, 'Pack of sugar': 28797, 'Army crackers': 14073, 'Salty Dog beef sausage': 18079, 'Can of sprats': 13864, 'Slickers chocolate bar': 9955, 'Pack of instant noodles': 7641, 'Alyonka chocolate bar': 22981, 'Can of Ice Green tea': 6039, 'Pack of apple juice': 7821, 'Pack of milk': 21236, 'Can of Hot Rod energy drink': 11778, 'Pack of Grand juice': 10100, 'Can of RatCola soda': 17156, 'Bottle of Fierce Hatchling moonshine': 266214, 'Bottle of Pevko Light beer': 23924, 'Can of Max Energy energy drink': 11864, 'Canister with purified water': 188675, 'Emergency Water Ration': 18172, 'Bottle of Tarkov

In [ ]:
for item in provisions_price_dict.items():
    print(item)

In [ ]:
full_dict = {}
for key, price_24h_avg in provisions_price_dict.items():
    energy, hydration = items_dict[key]
    full_dict[key] = [price_24h_avg, energy, hydration]
    print(f"name: {key}, price: {price_24h_avg}, energy: {energy}, hydration: {hydration}")


In [9]:
# Set column names
columns = ['name', 'price_24h_avg', 'hydration', 'energy']

df = pd.DataFrame.from_dict(full_dict, orient='index', columns=['price', 'energy', 'hydration']).reset_index()

# Rename the 'index' column to 'name'
df = df.rename(columns={'index': 'name'})


================================================================================================================================
DF ready
================================================================================================================================

In [ ]:
for name, attrs in full_dict.items():
    print(f"name: {name}, price: {attrs[0]}, energy: {attrs[1]}, hydration: {attrs[2]}")

In [21]:
def best_to_consume(current_energy, current_hydration, min_energy, min_hydration, df):
    """
    Returns the best item to consume based on the current energy and hydration levels.
    """
    # Calculate the difference between current energy and hydration levels and 

    dp_arr = [[float('inf') for i in range(total_hydration_columns)] for j in range(total_energy_rows)]

current_energy, current_hydration, min_energy, min_hydration = 0, 0, 100, 100

energy_diff = min_energy - current_energy
hydration_diff = min_hydration - current_hydration
total_energy_rows = energy_diff + 100
total_hydration_columns = hydration_diff + 100

dp_arr = [[float('inf') for i in range(total_hydration_columns)] for j in range(total_energy_rows)]
dp_arr[0][0] = 0
dp_provisions_arr = [[[] for i in range(total_hydration_columns)] for j in range(total_energy_rows)]
#dp_provisions_arr[0][0] = []
valid_start_points = [(0,0)]

while valid_start_points:
    start_energy_row, start_hydration_column = valid_start_points.pop(0)
    #print(f"trying starting point (e,h): {start_energy_row}, {start_hydration_column}")

    if start_energy_row >= energy_diff and start_hydration_column >= hydration_diff:
        #print(f"  starting point too far, removing (e,h): {start_energy_row}, {start_hydration_column}")
        continue

    for name, price, energy, hydration in df.values:


        #print(f"  starting point valid, jumping from (e,h): {start_energy_row}, {start_hydration_column}")
        result_energy_row = start_energy_row + energy
        result_hydration_column = start_hydration_column + hydration

        #check for out of bounds/negative effects
        if result_energy_row >= total_energy_rows or result_hydration_column >= total_hydration_columns or result_energy_row < 0 or result_hydration_column < 0:
            #print(f"  provision lands out of bounds, skipping (e,h): {result_energy_row}, {result_hydration_column}. name: {name}, price: {price}, energy: {energy}, hydration: {hydration}")
            continue
        
        curr_combo_price = dp_arr[start_energy_row][start_hydration_column] + price
        curr_combo_provisions = dp_provisions_arr[start_energy_row][start_hydration_column] + [name]
        if dp_arr[result_energy_row][result_hydration_column] > curr_combo_price:
            dp_arr[result_energy_row][result_hydration_column] = curr_combo_price
            dp_provisions_arr[result_energy_row][result_hydration_column] = curr_combo_provisions

            valid_start_points.append((result_energy_row, result_hydration_column))

            for energy_row in range(0, result_energy_row):
                for hydration_column in range(0, result_hydration_column):
                    if energy_row < start_energy_row and hydration_column < start_hydration_column:
                        continue
                    if dp_arr[energy_row][hydration_column] > curr_combo_price:
                        dp_arr[energy_row][hydration_column] = curr_combo_price
                        dp_provisions_arr[energy_row][hydration_column] = curr_combo_provisions
print(f"starting energy: {current_energy}, starting hydration: {current_hydration}")
print(f"target energy: {min_energy}, target hydration: {min_hydration}")
print(f"need to consume {energy_diff} energy and {hydration_diff} hydration")
print(f"minimal price price to reach the goal: {dp_arr[energy_diff][hydration_diff]}")
print(f"achieved by: ")
for provision_name in dp_provisions_arr[energy_diff][hydration_diff]:
    print(f"    name: {provision_name}, price: {full_dict[provision_name][0]}, energy: {full_dict[provision_name][1]}, hydration: {full_dict[provision_name][2]}")
print(f"final energy: {current_energy + sum([full_dict[provision_name][1] for provision_name in dp_provisions_arr[energy_diff][hydration_diff]])}")
print(f"final hydration: {current_hydration + sum([full_dict[provision_name][2] for provision_name in dp_provisions_arr[energy_diff][hydration_diff]])}")

        




starting energy: 0, starting hydration: 0
target energy: 100, target hydration: 100
need to consume 100 energy and 100 hydration
minimal price price to reach the goal: 48843
achieved by: 
    name: Iskra ration pack, price: 23087, energy: 80, hydration: 0
    name: Aquamari water bottle with filter, price: 25756, energy: 20, hydration: 100
final energy: 100
final hydration: 100


In [23]:
def best_to_consume(current_energy, current_hydration, goal_energy, goal_hydration, reference_dict):
    """
    Returns the best item to consume based on the current energy and hydration levels.
    """

    #set up the re-used variables and the DP array
    energy_difference = goal_energy - current_energy
    hydration_difference = goal_hydration - current_hydration
    total_energy_rows = energy_difference + 100
    total_hydration_columns = hydration_difference + 100


    dp_arr = [[float('inf') for i in range(total_hydration_columns)] for j in range(total_energy_rows)]
    dp_arr[0][0] = 0
    dp_provisions_arr = [[[] for i in range(total_hydration_columns)] for j in range(total_energy_rows)]
    valid_start_points = [(0,0)]


    while valid_start_points:
        start_energy_row, start_hydration_column = valid_start_points.pop(0)
        #print(f"trying starting point (e,h): {start_energy_row}, {start_hydration_column}")

        if start_energy_row >= energy_difference and start_hydration_column >= hydration_difference:
            #print(f"  starting point too far, removing (e,h): {start_energy_row}, {start_hydration_column}")
            continue
        
        for name, provision_attributes in reference_dict.items():
            price, energy, hydration = provision_attributes[0], provision_attributes[1], provision_attributes[2]
        
            #print(f"  starting point valid, jumping from (e,h): {start_energy_row}, {start_hydration_column}")
            result_energy_row = start_energy_row + energy
            result_hydration_column = start_hydration_column + hydration

            #check for out of bounds/negative effects
            if result_energy_row >= total_energy_rows or result_hydration_column >= total_hydration_columns or result_energy_row < 0 or result_hydration_column < 0:
                #print(f"  provision lands out of bounds, skipping (e,h): {result_energy_row}, {result_hydration_column}. name: {name}, price: {price}, energy: {energy}, hydration: {hydration}")
                continue
            
            curr_combo_price = dp_arr[start_energy_row][start_hydration_column] + price
            if dp_arr[result_energy_row][result_hydration_column] > curr_combo_price:
                dp_arr[result_energy_row][result_hydration_column] = curr_combo_price
                curr_combo_provisions = dp_provisions_arr[start_energy_row][start_hydration_column] + [name]
                dp_provisions_arr[result_energy_row][result_hydration_column] = curr_combo_provisions

                valid_start_points.append((result_energy_row, result_hydration_column))

                for energy_row in range(0, result_energy_row):
                    for hydration_column in range(0, result_hydration_column):
                        if energy_row < start_energy_row and hydration_column < start_hydration_column:
                            continue
                        if dp_arr[energy_row][hydration_column] > curr_combo_price:
                            dp_arr[energy_row][hydration_column] = curr_combo_price
                            dp_provisions_arr[energy_row][hydration_column] = curr_combo_provisions
    print(f"starting energy: {current_energy}, starting hydration: {current_hydration}")
    print(f"target energy: {goal_energy}, target hydration: {goal_hydration}")
    print(f"need to consume {energy_difference} energy and {hydration_difference} hydration")
    print(f"minimal price price to reach the goal: {dp_arr[energy_difference][hydration_difference]}")
    print(f"achieved by: ")
    for provision_name in dp_provisions_arr[energy_difference][hydration_difference]:
        print(f"    name: {provision_name}, price: {reference_dict[provision_name][0]}, energy: {reference_dict[provision_name][1]}, hydration: {reference_dict[provision_name][2]}")
    print(f"final energy: {current_energy + sum([reference_dict[provision_name][1] for provision_name in dp_provisions_arr[energy_difference][hydration_difference]])}")
    print(f"final hydration: {current_hydration + sum([reference_dict[provision_name][2] for provision_name in dp_provisions_arr[energy_difference][hydration_difference]])}")

        


In [ ]:
best_to_consume(22, 13, 90, 90, full_dict)

In [1]:
from utils import *

In [22]:
import requests
from bs4 import BeautifulSoup


# Send a GET request to the URL and get the HTML content
response = requests.get(url)
html_content = response.content
page = open('C:/Users/danil/Desktop/CLASSES/tarkov_thing.html')
# Create a BeautifulSoup object
soup = BeautifulSoup(page.read())

# Find all anchor tags (links) that contain the specified substring
links = soup.find_all('a', href=lambda href: href and '/item/' in href)

# Extract and print the href attribute from each matching link
for link in set(links):
    print(link.get('href'))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 278443: character maps to <undefined>